# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb langchain langchain_community langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 79.9 MB/s eta 0:00:0

In [2]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
from google.colab import userdata

# ✅ Retrieve and manually inject the secret into environment
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")

# ✅ Confirm
print("OPENAI_API_KEY is set:", os.environ.get("OPENAI_API_KEY") is not None)

Mounted at /content/drive
OPENAI_API_KEY is set: True


In [4]:
llm = OpenAI(temperature=0)

In [9]:
# ✅ Core packages for LangChain + community tools + Chroma
!pip install -qU langchain langchain_community langchain_openai chromadb

# Load Shakespeare Sonnets from file
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("/content/drive/MyDrive/Colab Notebooks/W7/d5/lab-agent-vector-store-main/sonnets.txt")  # <- Make sure you've downloaded this
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)


In [10]:
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import WebBaseLoader

llm = OpenAI(temperature=0)

# Create embeddings
embeddings = OpenAIEmbeddings()

# Create vector store for Sonnets
sonnets_db = Chroma.from_documents(texts, embeddings, collection_name="shakespeare-sonnets")
sonnets_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=sonnets_db.as_retriever())

# Load and vectorize Ruff docs
ruff_docs = WebBaseLoader("https://beta.ruff.rs/docs/faq/").load()
ruff_chunks = text_splitter.split_documents(ruff_docs)
ruff_db = Chroma.from_documents(ruff_chunks, embeddings, collection_name="ruff-docs")
ruff_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever())


In [5]:
# from pathlib import Path

# relevant_parts = []
# for p in Path(".").absolute().parts:
#     relevant_parts.append(p)
#     if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
#         break
# doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [8]:
# loader = TextLoader(doc_path)
# documents = loader.load()
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# texts = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings()
# docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [ ]:
# state_of_union = RetrievalQA.from_chain_type(
#     llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
# )

In [ ]:
# from langchain_community.document_loaders import WebBaseLoader

In [ ]:
# loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [ ]:
# docs = loader.load()
# ruff_texts = text_splitter.split_documents(docs)
# ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
# ruff = RetrievalQA.from_chain_type(
#     llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
# )

Created a chunk of size 2134, which is longer than the specified 1000
Created a chunk of size 2470, which is longer than the specified 1000
Created a chunk of size 1259, which is longer than the specified 1000
Created a chunk of size 1636, which is longer than the specified 1000


## Create the Agent

In [11]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [12]:
from langchain.agents import Tool, initialize_agent, AgentType

tools = [
    Tool(
        name="Shakespeare Sonnets QA",
        func=sonnets_qa.run,
        description="Use this to answer questions about Shakespeare's Sonnets. Input must be a complete question.",
    ),
    Tool(
        name="Ruff Python QA",
        func=ruff_qa.run,
        description="Use this to answer questions about the Ruff Python linter. Input must be a complete question.",
    ),
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


<ipython-input-12-487e4eda2dee>:16: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [ ]:
# # Construct the agent. We will use the default agent type here.
# # See documentation for a full list of options.
# agent = initialize_agent(
#     tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
# )

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [14]:
agent.invoke("What is the main theme of Sonnet 18?")




> Entering new AgentExecutor chain...
 The main theme of Sonnet 18 is likely related to love or beauty.
Action: Shakespeare Sonnets QA
Action Input: "What is the main theme of Sonnet 18?"
Observation:  The main theme of Sonnet 18 is the speaker's comparison of the subject to a summer's day and the idea that the subject's beauty will never fade.
Thought: This aligns with my initial thought that the theme is related to love and beauty.
Final Answer: The main theme of Sonnet 18 is the speaker's comparison of the subject to a summer's day and the idea that the subject's beauty will never fade.

> Finished chain.


{'input': 'What is the main theme of Sonnet 18?',
 'output': "The main theme of Sonnet 18 is the speaker's comparison of the subject to a summer's day and the idea that the subject's beauty will never fade."}

In [15]:
agent.invoke("How is Ruff different from flake8?")




> Entering new AgentExecutor chain...
 Ruff is a linter, so it probably has different rules and checks compared to flake8.
Action: Ruff Python QA
Action Input: "How is Ruff different from flake8?"
Observation:  Ruff is different from flake8 in that it uses different rule codes and prefixes, implements more rules, and does not support custom lint rules. It also has some minor differences in its rule set and supports automatic fixing of lint violations. Additionally, Ruff is designed to be used alongside a type checker, while flake8 supports plugins and custom rules.
Thought: I now know the final answer.
Final Answer: Ruff is different from flake8 in its rule codes, rule set, support for custom rules, and integration with type checkers.

> Finished chain.


{'input': 'How is Ruff different from flake8?',
 'output': 'Ruff is different from flake8 in its rule codes, rule set, support for custom rules, and integration with type checkers.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [ ]:
tools = [
    Tool(
        name="Shakespeare Sonnets QA",
        func=sonnets_qa.run,
        description="QA over Shakespeare's Sonnets",
        return_direct=True
    ),
    Tool(
        name="Ruff Python QA",
        func=ruff_qa.run,
        description="QA over Ruff documentation",
        return_direct=True
    )
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.invoke("What is Sonnet 116 about?")


In [ ]:
# agent = initialize_agent(
#     tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
# )

In [16]:
agent.invoke("Does Ruff support Jupyter Notebooks?")




> Entering new AgentExecutor chain...
 I should check the documentation for Ruff Python QA
Action: Ruff Python QA
Action Input: Does Ruff support Jupyter Notebooks?
Observation:  Yes, Ruff has built-in support for linting and formatting Jupyter Notebooks. It also integrates with nbQA, a tool for running linters and code formatters over Jupyter Notebooks.
Thought: I should also check the Ruff Python QA GitHub repository for any additional information
Action: Ruff Python QA
Action Input: Additional information on Jupyter Notebook support
Observation:  Yes, Ruff has built-in support for linting and formatting Jupyter Notebooks. It also integrates with nbQA, a tool for running linters and code formatters over Jupyter Notebooks. You can run Ruff over a notebook by installing Ruff and nbqa, and then using the command "nbqa ruff Untitled.ipynb".
Thought: I now know the final answer
Final Answer: Yes, Ruff does support Jupyter Notebooks.

> Finished chain.


{'input': 'Does Ruff support Jupyter Notebooks?',
 'output': 'Yes, Ruff does support Jupyter Notebooks.'}

In [17]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.invoke(
    "What does Shakespeare say about time in Sonnet 60? Is time also mentioned in the Ruff documentation?"
)




> Entering new AgentExecutor chain...
 I should use the Shakespeare Sonnets QA tool to find out what Shakespeare says about time in Sonnet 60, and then use the Ruff Python QA tool to see if time is mentioned in the Ruff documentation.
Action: Shakespeare Sonnets QA
Action Input: "What does Shakespeare say about time in Sonnet 60?"
Observation:  In Sonnet 60, Shakespeare describes time as a destructive force that takes away youth, beauty, and love. He compares time to the waves eroding the shore and to a scythe that mows down everything in its path. Despite this, the speaker hopes that his words will stand the test of time and continue to praise the worth of his subject.
Thought: Now I should use the Ruff Python QA tool to see if time is mentioned in the Ruff documentation.
Action: Ruff Python QA
Action Input: "Is time mentioned in the Ruff documentation?"
Observation:  Yes, time is mentioned in the Ruff documentation.
Thought: I now know the final answer.
Final Answer: Yes, time is m

{'input': 'What does Shakespeare say about time in Sonnet 60? Is time also mentioned in the Ruff documentation?',
 'output': "Yes, time is mentioned in both Shakespeare's Sonnet 60 and the Ruff documentation."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [22]:
from langchain.agents import Tool, initialize_agent, AgentType

tools = [
    Tool(
        name="Shakespeare Sonnets QA",
        func=sonnets_qa.run,
        description="Use this to answer questions about Shakespeare's Sonnets. Input should be a full question, no vague references.",
    ),
    Tool(
        name="Ruff Python QA",
        func=ruff_qa.run,
        description="Use this to answer questions about the Ruff Python linter. Input should be a full question, no vague references.",
    ),
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [20]:
# # Construct the agent. We will use the default agent type here.
# # See documentation for a full list of options.
# agent = initialize_agent(
#     tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
# )

In [23]:
agent.invoke(
    "What does Shakespeare say about time in Sonnet 60? Does Ruff also mention time in any way, like linting speed?"
)




> Entering new AgentExecutor chain...
 I should use the Shakespeare Sonnets QA tool to find out what Shakespeare says about time in Sonnet 60, and then use the Ruff Python QA tool to see if it mentions time in any way.
Action: Shakespeare Sonnets QA
Action Input: "What does Shakespeare say about time in Sonnet 60?"
Observation:  In Sonnet 60, Shakespeare describes time as a destructive force that takes away youth, beauty, and love. He compares time to the waves eroding the shore and to a scythe that mows down everything in its path. Despite this, the speaker hopes that his words will stand the test of time and continue to praise the worth of his subject.
Thought: Now I should use the Ruff Python QA tool to see if it mentions time in any way.
Action: Ruff Python QA
Action Input: "Does Ruff mention time in any way?"
Observation:  No, Ruff does not mention time in any way.
Thought: I now know the final answer.
Final Answer: Shakespeare describes time as a destructive force in Sonnet 60,

{'input': 'What does Shakespeare say about time in Sonnet 60? Does Ruff also mention time in any way, like linting speed?',
 'output': 'Shakespeare describes time as a destructive force in Sonnet 60, while Ruff does not mention time at all.'}